## Import necessary modules (Run this once at the beginning only)

In [1]:
# Import modules
import matplotlib.pyplot as plt
import numpy as np
# import tensorflow as tf
import pandas as pd
# from tensorflow import keras
# from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
# from keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
# from keras.layers import CuDNNLSTM
# from keras.models import Sequential
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# Initial figure settings
plt.rcParams['figure.figsize'] = [16,8]
plt.rcParams.update({'font.size': 18})
# Initial model training settings
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Background classes and functions (Run this once at the beginning only)

In [3]:
# EDA class
class READ:
  def __init__(self,name):
    self.name = name

  def read_daily_hourly(self):
    dataframe = pd.ExcelFile(self.name)
    # Read daily data set
    daily = pd.read_excel(dataframe, 'Daily Data', usecols="A:K")
    daily = daily.rename(columns={'Unnamed: 0': 'Date'})
    daily.sort_values('Date')
    daily['Date'] = pd.to_datetime(daily['Date'])
    daily['Day'] = daily['Date'].dt.date
    
    # Read hourly data set
    hourly = pd.read_excel(dataframe, 'Hourly Data')
    hourly.sort_values('timestamp')
    hourly['timestamp'] = pd.to_datetime(hourly['timestamp'])
    hourly['Day'] = hourly['timestamp'].dt.date
    hourly['Hour'] = hourly['timestamp'].dt.time
    return daily, hourly

class EDA:
  def __init__(self,data):
    self.data = data

  def data_sourcing(self):
    zeros = np.argwhere(self.data == 0)
    nans = np.argwhere(np.isnan(self.data))
    return [zeros, nans]

  def data_cleaning(self,interpolate=False):
    zeros, nans = self.data_sourcing()
    if (interpolate):
      pass
    else:
      remove = np.unique(np.concatenate([zeros[:,0], nans[:,0]]))
      return np.delete(self.data, remove, axis=0)

  def manipulation_analysis(self,frequency):
    diff_usd = [t - s for s, t in zip(self.data[::frequency], self.data[frequency::frequency])]
    diff_per = [(t - s)/s*100 for s, t in zip(self.data[::frequency], self.data[frequency::frequency])]
    sort_diff_usd = np.sort(diff_usd)
    sort_diff_per = np.sort(diff_per)
    abs_diff_usd = np.sort(np.absolute(diff_usd))
    abs_diff_per = np.sort(np.absolute(diff_per))
    upper = np.mean(abs_diff_per[-20:]) + np.std(abs_diff_per[-20:])
    lower = np.mean(abs_diff_per[-20:]) - np.std(abs_diff_per[-20:])
    return [lower, upper]

In [4]:
# Calculate Market Confidence Level
def calc_confidence_level(last,actual,mani_limits):
  lower = mani_limits[0]
  upper = mani_limits[1]
  diff = abs(last - actual)/last*100
  if (diff <= lower):
    return 100
  elif (diff >= upper):
    return 0
  else:
    return 100 - (diff-lower)/(upper-lower)*100

In [5]:
# Calculate Model Confidence Level
def calc_model_confidence(predicted,actual):
  diff = abs(predicted - actual)/actual*100
  return 100 - diff

In [6]:
# SVD Linear Regression Model
class SVD:
  def __init__(self):
    pass

  def split_validation_data(self,A,b,t):
    # p = np.random.permutation(len(A))
    # A = A[p]
    # b = b[p]
    A_train = A[:-t]
    b_train = b[1:-t+1]
    A_test = A_train[:-t]
    b_test = b_train[:-t]
    return A_train, b_train, A_test, b_test
    
  def linear_regression(self,A_train,b_train):
    U, S, VT = np.linalg.svd(A_train,full_matrices=False)
    x = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ b_train
    return x

  def get_svd_error(self,A_test,b_test,x):
    error = mean_squared_error(A_test @ x, b_test)
    return error

  def normalize_data(self,A):
    A_mean = np.mean(A,axis=0)
    A_mean = A_mean.reshape(-1,1)
    A2 = A - np.ones((A.shape[0],1)) @ A_mean.T
    for j in range(A.shape[1]):
      A2std = np.std(A2[:,j])
      A2[:,j] = A2[:,j]/A2std
    return A2

In [7]:
# LSTM model
def lstm(data,seq_length,per_train):
  # Scale data
  scaler = MinMaxScaler()
  scaled_btc_price = scaler.fit_transform(data.reshape(-1,1))
  np.isnan(scaled_btc_price).any()

  # Create LSTM sequences
  def to_sequences(data, n):
    d = []
    for index in range(len(data) - n):
      d.append(data[index: index + n])
    return np.array(d)

  def preprocess(data_raw, n, per_train):
    data = to_sequences(data_raw, n)
    num_train = int(per_train * data.shape[0])
    A_train = data[:num_train, :-1, :]
    b_train = data[:num_train, -1, :]
    A_test = data[num_train:, :-1, :]
    b_test = data[num_train:, -1, :]
    return A_train, b_train, A_test, b_test

  A_train, b_train, A_test, b_test = preprocess(scaled_btc_price, seq_length, per_train)

  # Model
  DROPOUT = 0.2
  WINDOW_SIZE = seq_length - 1
  model = keras.Sequential()
  model.add(Bidirectional(CuDNNLSTM(WINDOW_SIZE, return_sequences=True),
    input_shape=(WINDOW_SIZE, A_train.shape[-1])))
  model.add(Dropout(rate=DROPOUT))
  model.add(Bidirectional(CuDNNLSTM((WINDOW_SIZE * 2), return_sequences=True)))
  model.add(Dropout(rate=DROPOUT))
  model.add(Bidirectional(CuDNNLSTM(WINDOW_SIZE, return_sequences=False)))
  model.add(Dense(units=1))
  model.add(Activation('linear'))

  # Train
  model.compile(
    loss='mean_squared_error',
    optimizer='adam'
  )
  BATCH_SIZE = 64
  history = model.fit(
    A_train,
    b_train,
    epochs=50,
    batch_size=BATCH_SIZE,
    shuffle=False,
    validation_split=0.1
  )

  # Model evaluation
  model.evaluate(A_test, b_test)

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

  # Prediction
  y_hat = model.predict(A_test)
  y_test_inverse = scaler.inverse_transform(b_test)
  y_hat_inverse = scaler.inverse_transform(y_hat)

  plt.plot(y_test_inverse, label="Actual Price", color='green')
  plt.plot(y_hat_inverse, label="Predicted Price", color='red')
  plt.title('Bitcoin price prediction')
  plt.xlabel('Time [days]')
  plt.ylabel('Price')
  plt.legend(loc='best')
  plt.show()

  return model

In [8]:
# Curve fit for hourly
def poly_fit(x,a,b,c,d):
  return a * x**3 + b * x**2 + c * x + d
def sin_fit(x,a,b,c,d,e):
  return a * np.sin(b - x) + c * x**2 + d * x + e

class CFH:
  def __init__(self,hourly):
    self.hourly = hourly

  def get_hourly_from_daily(self,daily,index):
    day = daily.loc[index]['Day']
    hours = self.hourly.loc[self.hourly['Day'] == day]
    return hours['BTC PRICE- USD'].to_numpy()

  def curve_fitting(self,data):
    x = np.arange(len(data)-1,47)
    y = np.arange(48)
    diff = np.setdiff1d(y,x)
    popt1, _ = curve_fit(poly_fit,diff,data)
    a,b,c,d = popt1
    predicted = poly_fit(diff,a,b,c,d)
    mse1 = mean_squared_error(predicted,data)
    popt2, _ = curve_fit(sin_fit,diff,data)
    a,b,c,d,e = popt2
    predicted = sin_fit(diff,a,b,c,d,e)
    mse2 = mean_squared_error(predicted,data)
    if (mse1 >= mse2):
      a,b,c,d,e = popt2
      return sin_fit(np.arange(24,48),a,b,c,d,e)
    else:
      a,b,c,d = popt1 
      return poly_fit(np.arange(24,48),a,b,c,d)

  def calc_local_min(self,data):
    return min(data), np.argmin(data)

  def calc_local_max(self,data):
    return max(data), np.argmax(data)

In [9]:
# Class for processing data and printing during validation
class TRADING:
  def __init__(self,last,predicted,conf_market,conf_model):
    self.last = last
    self.predicted = predicted
    self.conf_market = conf_market
    self.conf_model = conf_model

  def print_current_state(self):
    print ("  Current actual price: ", self.last)
    print ("  Predicted next price: ", self.predicted)
    print ("  Percentage difference: ", (self.predicted-self.last)/self.last*100)
    print ("  Confidence Levels for Market: %d and Model: %d" % (self.conf_market, self.conf_model))

  def add_current_to_train(self,A_tt,b_tt,A_new,b_new):
    print ("  Adding current data to the list and removing the oldest.")
    A_tt = np.delete(A_tt,(0),axis=0)
    b_tt = np.delete(b_tt,0)
    A_tt = np.append(A_tt,[A_new],axis=0)
    b_tt = np.append(b_tt,b_new)
    return A_tt, b_tt
 
  # def calc_hourly_decision():

  def calc_daily_decision(self,open_trade):
    if (self.conf_market >= 50) and (self.conf_model >= 75):
      print ("  Market and Model are reliable.")
      if (self.predicted > self.last*1.0015):
        if (open_trade == -1):
          decision = [1,1]
        elif (open_trade == 0):
          decision = [0,1]
        else:
          decision = [0,0]
      elif (self.predicted < self.last*0.9985):
        if (open_trade == 1):
          decision = [1,-1]
        elif (open_trade == 0):
          decision = [0,-1]
        else:
          decision = [0,0]
      else:
        decision = [0,0]
    else:
      print ("  Either Market or Model is not reliable.")
      if (open_trade != 0):
        decision = [1,2]
      else:
        decision = [0,2]
    return decision

# Calculate profit factor
def calc_profit(price_list,decisions):
  loss=0
  profit=0
  for i in range(int(len(price_list)/2)):
    t = price_list[2*i+1]-price_list[2*i]
    d = decisions[i]
    if (t*d < 0):
      loss+=abs(t*d)
    else:
      profit+=abs(t*d)
  if (loss != 0):
    print ("Profit Factor in Trade Term:",profit/loss)
  else:
    print ("No Losses. Total profit is: ", profit)
 
# Print Decision function
def print_decision(decision,hour):
  c = decision[0]
  d = decision[1]
  if (c == 0) and (d == 1):
    print ("  Decision is Open New Trade as Buy at Hour: ", hour)
  elif (c == 0) and (d == -1):
    print ("  Decision is Open New Trade as Sell at Hour: ", hour)
  elif (c == 0) and (d == 0):
    print ("  Decision is Hold")
  elif (c == 0) and (d == 2):
    print ("  Decision is Hold and Re-train the model")
  elif (c == 1) and (d == 1):
    print ("  Decision is Close last Trade and Open new as Buy at Hour: ", hour)
  elif (c == 1) and (d == -1):
    print ("  Decision is Close last Trade and Open new as Sell at Hour: ", hour)
  elif (c == 1) and (d == 0):
    print ("  Decision is Close last Trade at Hour: ", hour)
  elif (c == 1) and (d == 2):
    print ("  Decision is Close last Trade and Re-train the model at Hour: ", hour)

In [10]:
def curve_fit_hourly(hourly,daily,nt,last,predicted,i,MinMax):
  # Curve fit hourly data to find local min
  cfh = CFH(hourly)
  hours = cfh.get_hourly_from_daily(daily,nt+i-1) # Get previous day hourly data
  next_hours = cfh.get_hourly_from_daily(daily,nt+i) # Get next day hourly data
  hours = np.append(hours,last) # add current price to previous day hourly data
  hours = np.append(hours,predicted) # add prediction to previous day hourly data
  for j in range(len(next_hours)):
    curve = cfh.curve_fitting(hours)
    if (MinMax == -1):
      value, index = cfh.calc_local_min(curve)
    elif (MinMax == 1):
      value, index = cfh.calc_local_max(curve)
    if (index == j):
      return value, j
    else:
      hours = np.insert(hours,-1,next_hours[j])
  return value, j

# Initial Input File Reading (Run this once at the beginning only)
Please input the Bitcoin data Excel file using the same format as the one provided during the model development stages. The current code expects an Excel file named "OnChainvs_Price_proj712.xlsx" file read from the user's Google Drive. 

In [11]:
# Read data
name = '/content/drive/MyDrive/OnChainvs_Price_proj712.xlsx'
dataframe = READ(name)
daily, hourly = dataframe.read_daily_hourly()

In [12]:
# EDA for the daily data
arr_daily = daily.iloc[:,1:11].to_numpy()
eda = EDA(arr_daily)
arr_daily = eda.data_cleaning()
# Determine manipulation values
btc_daily = arr_daily[:,0]
eda = EDA(btc_daily)
daily_mani = eda.manipulation_analysis(1)
# print ("Daily Manipulation Limits: ", daily_mani)

# EDA for the daily data
arr_hourly = hourly.iloc[:,1:7].to_numpy()
eda = EDA(arr_hourly)
arr_hourly = eda.data_cleaning()
# Determine manipulation values
btc_hourly = arr_hourly[:,0]
eda = EDA(btc_hourly)
hourly_mani = eda.manipulation_analysis(4)
# print ("Hourly Manipulation Limits: ", hourly_mani)

In [13]:
# SVD Linear Regression for daily data
A = arr_daily[:-1,1:]
b = arr_daily[1:,0] 
test = 30
nt = A.shape[0]

# Split data into Train, Test and Dynamic Validation
svd = SVD()
A_train, b_train, A_test, b_test = svd.split_validation_data(A,b,test)
# A_train_norm = svd.normalize_data(A_train)
daily_x = svd.linear_regression(A_train,b_train)
daily_error = svd.get_svd_error(A_test,b_test,daily_x)

# A_tt = np.concatenate((A_train,A_test),axis=0)
# b_tt = np.concatenate((b_train,b_test))
# daily_x = svd.linear_regression(A_tt,b_tt)

# Calculate initial confidence level
predicted = A[-1] @ daily_x # last predicted price from the test data
actual = b[-1] # actual price for the same date
last = b[-2] # actual price for the previous date
conf_model = calc_model_confidence(predicted,actual) # initial confidence level for the model
conf_market = calc_confidence_level(last,actual,daily_mani) # initial confidence level for the market
print ("Initial Market confidence level: ", conf_market)
print ("Initial Model confidence level: ", conf_model)

# Initiate variables for the trading algorithm
price_list=[]
buy_indices=[]
sell_indices=[]
profits=[]
decisions=[]
dec_counter=np.array([0,0,0]) # initialize Buy/Sell/Hold counter
open_trade = 0
close = False
day = 0
hour = 0 # Change the time you wanna start from here

Initial Market confidence level:  100
Initial Model confidence level:  99.20205389821957


# Dynamic Trading Algorithm. 
Run the below every hour after updating the daily/hourly data in the Excel file. If you have Google Colab Pro+, you can use Schedule to automate this task.

In [21]:
# Read data for the next day
name = '/content/drive/MyDrive/OnChainvs_Price_proj712.xlsx'
dataframe = READ(name)
daily, hourly = dataframe.read_daily_hourly()
arr_daily = daily.iloc[:,1:11].to_numpy()
btc_daily = arr_daily[:,0]
arr_hourly = hourly.iloc[:,1:7].to_numpy()
btc_hourly = arr_hourly[:,0]
A = arr_daily[:,1:]
b = arr_daily[:,0]
if (hour == 24):
  hour = 0
  day += 1

print ("Day: ", day, " Hour: ", hour)
if (hour == 0): # Change the time you wanna start from here
  # Re-calculate the manipulation limits
  eda = EDA(b)
  mani = eda.manipulation_analysis(1)

  # Re-calculate the confidence levels
  last = b[-2]
  actual = b[-1]
  predicted = A[-2] @ daily_x
  conf_market = calc_confidence_level(last,actual,mani)
  conf_model = calc_model_confidence(predicted,actual)

  # Run dynamic simulation
  predicted = A[-1] @ daily_x
  last = b[-1]

  # Initiate validation class
  trade = TRADING(last,predicted,conf_market, conf_model)

  # Print current state
  trade.print_current_state()

  # Basic trading algorithm
  decision = trade.calc_daily_decision(open_trade)
  close = decision[0]
  dec = decision[1]
    
if (dec == 1):
  min_value, index = curve_fit_hourly(hourly,daily,nt,last,predicted,day,-1)
  if (index == hour):
    if (close):
      price_list.append(min_value)
      open_trade = 0
    price_list.append(min_value)
    decisions.append(dec)
    buy_indices.append([day,index])
    dec_counter = dec_counter+np.array([1,0,0])
    open_trade = 1
elif (dec == -1):
  max_value, index = curve_fit_hourly(hourly,daily,nt,last,predicted,day,+1)
  if (index == hour):
    if (close):
      price_list.append(max_value)
      open_trade = 0
    price_list.append(max_value)
    decisions.append(dec)
    sell_indices.append([day,index])
    dec_counter = dec_counter+np.array([0,1,0])
    open_trade = -1
elif (dec == 0):
  dec_counter = dec_counter+np.array([0,0,1])
elif (dec == 2):
  if (close):
    if (open_trade == -1):
      value, index = curve_fit_hourly(hourly,daily,nt,last,predicted,day,-1)
      if (index == hour):
        price_list.append(value)
        open_trade = 0
    elif (open_trade == 1):
      value, index = curve_fit_hourly(hourly,daily,nt,last,predicted,day,+1)
      if (index == hour):
        price_list.append(value)
        open_trade = 0
  last_dec = 0
  # Re-train the model
  svd = SVD()
  A_train, b_train, A_test, b_test = svd.split_validation_data(A,b,test)
  daily_x = svd.linear_regression(A_train,b_train)

# Print decision
print_decision(decision,index)
# Print previous decisions
print ("Buy Day and Hour so far: ", buy_indices)
print ("Sell Day and Hour so far: ", sell_indices)
print ("So far trade Open and Close prices: ", price_list)
dec_strings = []
for i in range(len(decisions)):
  if (decisions[i] == -1):
    dec_strings.append("Sell")
  elif (decisions[i] == 1):
    dec_strings.append("Buy")
print ("So far trade Buy or Sell decisions: ", dec_strings)
hour += 1

Day:  0  Hour:  7
  Decision is Open New Trade as Sell at Hour:  7
Buy Day and Hour so far:  []
Sell Day and Hour so far:  [[0, 7]]
So far trade Open and Close prices:  [21817.767646879965]
So far trade Buy or Sell decisions:  ['Sell']
